In [1]:
import os
import shutil
from threading import Thread
from glob import glob
from subprocess import Popen

import pandas as pd
import numpy as np

import datetime
import time

from features import get_features, perform_backtest

from twitter_stream import twitter_stream
from price_stream import price_stream
from utils.get_price_data import get_price
from utils.common_utils import get_root_dir, merge_csvs
from process import processor, get_sentiment, clean_further, add_keyword, clean_profile

Loading BokehJS ...

In [2]:
keywords = pd.read_csv('keywords.csv')

In [3]:
files = glob('data/test/*')

In [4]:
combined = merge_csvs(files)

In [5]:
df = pd.read_csv(combined)

In [6]:
df, user_info = processor(df)

/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/process.py:95: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/process.py:96: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [7]:
len(df)

1236040

In [8]:
len(user_info)

443808

In [9]:
df['Time'].dtype

dtype('int64')

In [10]:
df['ID'].dtype

dtype('int64')

In [11]:
df['Time'] = pd.to_datetime(df['Time'], unit='s')

In [12]:
df = add_keyword(df)

In [13]:
df['keyword'].value_counts(dropna=False)

invalid    865638
BTC        220742
ETH         50106
TRX         26803
XRP         26653
LTC         12940
EOS         10005
MIOTA        5494
LINK         4057
BCHABC       2323
XLM          2244
ADA          1556
ZEC          1418
XMR          1288
BNB          1098
BSV           982
XTZ           826
DASH          595
NEO           571
ETC           361
BTG           171
ZRX            88
OMG            37
ETP            26
SAN            18
Name: keyword, dtype: int64

In [14]:
df = df[df['keyword'] != 'invalid']

In [15]:
df = df.reset_index(drop=True)

In [16]:
df = get_sentiment(df)

/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/process.py:191: FutureWarning:

The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.



java -jar /home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/utils/SentiStrength.jar sentidata /home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/utils/SentiStrength_Data/ input /home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/data/temp/tweets


In [17]:
df['pos_neg'].value_counts(dropna=False)

 0    204810
 1     65900
 2     37442
-1     33652
-2     11371
 3      8994
-3      4450
 4      3608
-4       175
Name: pos_neg, dtype: int64

In [18]:
profilefile = 'data/cleaned_profile.csv'

In [19]:
if os.path.isfile(profilefile):
    user_info = pd.concat([user_info, pd.read_csv(profilefile)])
    user_info = clean_profile(user_info)

In [20]:
df = df.sort_values('Time')
df = df.set_index('Time')
df.index = df.index.ceil(freq='30Min')  
df = df.reset_index()

In [21]:
coin_name = 'BTC'

In [22]:
## VERIFYING TIME

In [23]:
pd.Timestamp('2019-08-09 06:01:00').ceil(freq='30Min') #test of ceil

Timestamp('2019-08-09 06:30:00')

In [24]:
price_df = pd.read_csv('data/price/{}.csv'.format(coin_name))

In [25]:
price_df['Time'] = pd.to_datetime(price_df['Time'])

In [26]:
price_df.head(20)

,Time,Open,High,Low,Close,Volume
0,2019-07-01 00:00:00,10854.10,10883.19,10840.10,10876.07,1.496492e+06
1,2019-07-01 00:01:00,10877.24,10885.57,10853.20,10881.44,1.099121e+06
2,2019-07-01 00:02:00,10880.16,10880.17,10835.00,10835.00,1.574824e+06
3,2019-07-01 00:03:00,10835.02,10840.00,10811.99,10828.56,1.719258e+06
4,2019-07-01 00:04:00,10826.60,10827.97,10775.00,10812.51,1.394439e+06
5,2019-07-01 00:05:00,10812.51,10826.74,10736.05,10736.05,2.634270e+06
6,2019-07-01 00:06:00,10735.78,10787.36,10719.00,10778.28,2.702564e+06
7,2019-07-01 00:07:00,10776.06,10840.00,10770.59,10830.20,1.606807e+06
8,2019-07-01 00:08:00,10825.82,10850.01,10805.70,10831.59,1.361710e+06
9,2019-07-01 00:09:00,10829.84,10870.00,10829.73,10862.75,1.045778e+06


In [27]:
get_price('BTC', duration='10Min').head(2) #uses the next 10 mins. Manually verifying that

,Time,Open,High,Low,Close,Volume
0,2019-07-01 00:10:00,10854.10,10885.57,10719.00,10862.75,1.663526e+07
1,2019-07-01 00:20:00,10858.68,11047.01,10812.17,10998.14,1.623573e+07


In [28]:
## VERIFIED TIME ##

In [29]:
price_df = get_price('BTC')
tweet_df = df[df['keyword'] == coin_name].drop('keyword', axis=1).reset_index(drop=True)

curr_start = df.iloc[0]['Time']
curr_end = df.iloc[-1]['Time']

In [30]:
features = get_features(tweet_df, price_df, coin_name, curr_start, curr_end) #verified

In [31]:
#Now verify cerebro

In [32]:
from features import perform_backtest

In [33]:
perform_backtest(coin_name, n_fast_par=24, n_slow_par=52, long_macd_threshold_par=2, long_per_threshold_par=1, 
                        long_close_threshold_par=1, short_macd_threshold_par=-1, short_per_threshold_par=-1, short_close_threshold_par=0.5,
                        initial_cash=10000, comission=0.1, df=features)

/home/warproxxx/Desktop/Projects/crypto-analysis-live/algorithm/features.py:277: FutureWarning:

using a dict with renaming is deprecated and will be removed in a future version



In [ ]:
os.remove('data/features/BTC.csv')